In [1]:
import json, os
from tqdm import tqdm
import pickle
from tqdm import trange
import math

#PyTorch用的包
import torch
import torch.nn as nn
import torch.optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# 自然语言处理相关的包
import re #正则表达式的包
import jieba #结巴分词包
from collections import Counter #搜集器，可以让统计词频更简单

#绘图、计算用的程序包
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# 设置随机种子保证可复现
import random
SEED = 729608
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
# os python hash seed, make experiment reproducable
os.environ['PYTHONHASHSEED'] = str(SEED)
# gpu algorithom 
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
# Generator SEED
Generator = torch.Generator()
Generator.manual_seed(SEED)

/home/charon/anaconda3/envs/qbw_base/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pth = './rumor_detection_data'
if not os.path.exists(pth):
    os.makedirs(pth)
good_file = os.path.join(pth, 'non_rumor.txt')
bad_file = os.path.join(pth, 'rumor.txt')
# 将文本中的标点符号过滤掉
def filter_punc(sentence):
    sentence = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？?、~@#￥%……&*（）：:；“”】》《-【\][]", "",sentence.strip())
    return sentence

# 扫描所有的文本，分词、建立词典，分出是谣言还是非谣言，is_filter可以过滤是否筛选掉标点符号
def Prepare_data(good_file, bad_file, is_filter = True, threshold=3):
    all_sentences = [] #存储所有的单词
    pos_sentences = [] #存储非谣言
    neg_sentences = [] #存储谣言
    with open(good_file, 'r', encoding='utf-8') as fr:
        for idx, line in enumerate(fr):
            if is_filter:
                #过滤标点符号
                line = filter_punc(line)
                if not idx: # 只打印第一个例子看看
                    print('分词前：', line)
            #分词
            words = jieba.lcut(line)
            if not idx: # 只打印第一个例子看看
                print('分词后：', words)
            if len(words) > 0:
                all_sentences.append(words)
                pos_sentences.append(words)
    count = sum([len(s) for s in all_sentences])
    print('{0} 包含 {1} 行, {2} 个词.'.format(good_file, idx+1, count))
    with open(bad_file, 'r', encoding='utf-8') as fr:
        for idx, line in enumerate(fr):
            if is_filter:
                line = filter_punc(line.strip())
            words = jieba.lcut(line)
            if len(words) > 0:
                all_sentences.append(words)
                neg_sentences.append(words)
    print('{0} 包含 {1} 行, {2} 个词.'.format(bad_file, idx+1, sum([len(s) for s in all_sentences])-count))

    return pos_sentences, neg_sentences,all_sentences


pos_sentences, neg_sentences,all_sentences = Prepare_data(good_file, bad_file, True, threshold=3)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


分词前： 最值得爱一生的五大星座排名第一名巨蟹座不畏艰辛患难与共第二名金牛座不离不弃第三名处女座不逃避困难第四名魔羯座一起吃苦第五名天蝎座真情守护被巨蟹座爱上是需要几世才能修来的好福气啊


Loading model cost 0.375 seconds.
Prefix dict has been built successfully.


分词后： ['最', '值得', '爱', '一生', '的', '五大', '星座', '排名', '第一名', '巨蟹座', '不畏', '艰辛', '患难与共', '第二名', '金牛座', '不离', '不弃', '第三名', '处女座', '不', '逃避', '困难', '第四名', '魔羯座', '一起', '吃苦', '第五名', '天蝎座', '真情', '守护', '被', '巨蟹座', '爱上', '是', '需要', '几世', '才能', '修来', '的', '好福气', '啊']
./rumor_detection_data/non_rumor.txt 包含 1849 行, 92943 个词.
./rumor_detection_data/rumor.txt 包含 1538 行, 78645 个词.


In [3]:
class Vocab: 
    def __init__(self,input_tokens=None,min_freq_threshold=10,special_token_tag=False):
        if special_token_tag: # whether to use pad,begin of sentance,end of sentence,unknown tokens
            self.pad,self.bos,self.eos,self.unk  = 0,1,2,3
            tokens = ['<pad>','<bos>','<eos>','<unk>'] # ?
            self.special_tokens = ['<pad>','<bos>','<eos>','<unk>']
        else:
            self.unk = 0
            tokens = ['<unk>']
            self.special_tokens = ['<unk>']
        
        # when load, we can just initialze a nearly empty Vocab class
        # then load .pkl file to give the value of token2idx,idx2token
        if input_tokens is None: 
            return 
            
        # hashmap to count the tokens(key:token,value:freq)
        assert len(input_tokens),'0 length is not allowed'
        if isinstance(input_tokens[0],list): 
            input_tokens = [token for sentence in input_tokens for token in sentence if token not in self.special_tokens]
            tokens_freq = Counter(input_tokens)
        else:
            tokens_freq = Counter(input_tokens) 
        tokens_freq = sorted(tokens_freq.items(),key=lambda x:x[0]) # sort the tokens_freq dict by dictionary order
        tokens_freq = sorted(tokens_freq,key=lambda x:x[1],reverse=True) # sort the tokens_freq dict by freq order
        tokens_freq = dict(tokens_freq)
        # print(tokens_freq)
        
        # establish two hashmaps to transform index to token or reverse transform
        self.idx2token = []
        self.token2idx = dict()

        # filter the tokens whose freq is letter than min_freq_threshold
        tokens += list(filter(lambda x:tokens_freq[x]>min_freq_threshold,tokens_freq.keys()))
        # tokens += [token for token,freq in tokens_freq.items() if freq>=min_freq_threshold]

        # add the token to the two hashmaps
        i = 0 # 0 index is unknown token
        for token in tokens:
            self.idx2token.append(token)
            self.token2idx[token] = i
            i += 1
        assert len(self.idx2token) == len(self.token2idx),(len(self.idx2token),len(self.token2idx))
    
    def __len__(self):
        return len(self.idx2token)
    
    def __getitem__(self,tokens): 
        '''
        input : tokens(single char or list/tuple) 
        output : idx(then torch.nn.Embedding automatically transform to one-hot code)
        '''
        if (not isinstance(tokens,tuple)) and (not isinstance(tokens,list)):
            return self.token2idx.get(tokens,self.unk)
        else:
            # recursive call the __getitem__,until the token is a single char
            # use this strategy, we can process higher dim tensor such as :
            # [[a,b,c,...]](shape:[n1,n2]) or even [[[a,b,c],[d,e,f]]](shape:[n1,n2,n3])
            # the return shape is the same as the input
            return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self,indices):
        '''
            input the indices
            output the corresponding tokens
        '''
        if (not isinstance(indices,tuple)) and (not isinstance(indices,list)):
            return self.idx2token[indices]
        else:
            return [self.to_tokens(index) for index in indices]


In [4]:
def subsample(sentences,vocab):
    '''
    subsample the sences to reduce the impace of the much high-frequence but less-message tokens like "the","a","in" and so on
    '''
    # 排除未知词元'<unk>'
    sentences_flatten = [token for line in sentences for token in line if vocab[token] != vocab.unk]
    counter = Counter(sentences_flatten)
    num_tokens = sum(counter.values()) # the sum number of the words/tokens

    def keep_func(token,t=1e-4): # t is a hyper-parameter
        # when freq(token) > t,it will be definitely dropped,else the higher freq,the higher prob to be dropped
        freq = counter[token] / num_tokens
        return (random.uniform(0, 1) > max((1 - math.sqrt(t / freq),0)))

    subsampled = []
    for line in sentences:
        temp = []
        for token in line:
            if token in counter.keys() and keep_func(token):
                temp.append(token)
        subsampled.append(temp)
    return subsampled,counter

def compare_counts(token):
    return (f'"{token}"的数量：'
            f'之前={sum([l.count(token) for l in sentences])}, '
            f'之后={subsampled.count(token)}')

# get the centers and contexts token to do the skip-gram language model
def get_centers_and_contexts(corpus, max_window_size):
    """return the center token and the corresponding context token"""
    centers, contexts = [], []
    for line in corpus: # corpus shape: [num_sentence,num_tokens_per_sentence]
        # if length of sentence is less than 2, cannot compose the center-context pair
        if len(line) < 2:
            continue
        centers += line
        for i in range(len(line)):  # center_idx = i
            # window_size = random.randint(1, max_window_size) # randomly generate the window_size
            window_size = max(1,max_window_size)
            indices = list(range(max(0, i - window_size),
                                 min(len(line), i + 1 + window_size))) # get the context token in the window(both left and right)
            # remove the center token
            indices.remove(i)
            contexts.append([(line[idx],abs(idx-i)) for idx in indices])
    return centers, contexts

def coappearence_computation(vocab_size,centers,contexts,use_distance_weight=False):
    coappearence_matrix = torch.zeros((vocab_size,vocab_size))
    if use_distance_weight:
        coappearence_matrix_2 = torch.zeros((vocab_size,vocab_size))
    lc = len(centers)
    for i in range(lc):
        row = centers[i]
        columns = contexts[i]
        for col in columns:
            # print(col)
            coappearence_matrix[row,col[0]] += 1.
            if use_distance_weight:
                coappearence_matrix_2[row,col[0]] +=1. / col[1]
    if use_distance_weight:
        return coappearence_matrix,coappearence_matrix_2
    else:
        return coappearence_matrix



def batchify(data):
    '''
    if want to batchify the data, we need:
        - padding the data because the num_context_per_center is not the same
        - generate the mask so that the padding part will not be included in the loss_func computation
    '''
    max_len = max([len(c) for _,c,_ in data]) 
    centers,all_contexts,all_masks,all_labels = [],[],[],[]
    for i,item in enumerate(data):
        center,context,label = item
        centers.append(center)
        cur_len = len(context) 
        all_contexts.append(context + [0]*(max_len - cur_len))
        all_masks.append([1.]*cur_len + [0.]*(max_len-cur_len))
        all_labels.append(label + [1.]*(max_len-len(label))) # can't pad 0,because log(0) = -inf
    
    centers = torch.tensor(centers).reshape(-1,1)
    all_contexts = torch.tensor(all_contexts)
    all_masks = torch.tensor(all_masks)
    all_labels = torch.tensor(all_labels)
    return centers,all_contexts,all_masks,all_labels

def batchify2(data):
    '''
    if want to batchify the data, we need:
        - padding the data because the num_context_per_center is not the same
        - generate the mask so that the padding part will not be included in the loss_func computation
    '''
    max_len = max([len(c) for _,c,_,_ in data]) 
    centers,all_contexts,all_masks,all_labels,all_weights = [],[],[],[],[]
    for i,item in enumerate(data):
        center,context,label,weight = item
        centers.append(center)
        cur_len = len(context) 
        all_contexts.append(context + [0]*(max_len - cur_len))
        all_masks.append([1.]*cur_len + [0.]*(max_len-cur_len))
        all_labels.append(label + [1.]*(max_len-len(label))) # can't pad 0,because log(0) = -inf
        all_weights.append(weight + [1.]*(max_len-len(weight)))
    
    centers = torch.tensor(centers).reshape(-1,1)
    all_contexts = torch.tensor(all_contexts)
    all_masks = torch.tensor(all_masks)
    all_labels = torch.tensor(all_labels)
    all_weights = torch.tensor(all_weights)
    return centers,all_contexts,all_masks,all_labels,all_weights

In [5]:
def load_data_ptb(bs,max_window_size,sentences,use_distance_weight=False,load_vocab=True,save_dir=None):
    if load_vocab:
        vocab = Vocab()
        with open(os.path.join(save_dir,'token2idx.pkl'),'rb') as f:
            vocab.token2idx = pickle.load(f)
        with open(os.path.join(save_dir,'idx2token.pkl'),'rb') as f:
            vocab.idx2token = pickle.load(f)
    else:
        vocab = Vocab(sentences,min_freq_threshold=3,special_token_tag=False)
    print(f'词表大小为：{len(vocab)}')


    # subsampling 
    # subsampled, counter = subsample(sentences, vocab)

    subsampled = sentences

    # transfrom token to idx(getitem)
    corpus = [vocab[line] for line in subsampled]

    # generate center-context pair
    all_centers, all_contexts = get_centers_and_contexts(corpus, max_window_size)

    # compute the co-appearence matrix
    if use_distance_weight:
        coappearence_matrix,coappearence_matrix_2 = coappearence_computation(len(vocab),all_centers,all_contexts,use_distance_weight=use_distance_weight)
    else:
        coappearence_matrix = coappearence_computation(len(vocab),all_centers,all_contexts,use_distance_weight=use_distance_weight)

    class PTB_DATASET(Dataset):
        def __init__(self, centers, contexts,use_distance_weight,coappearence_matrix):
            assert len(centers) == len(contexts) ,f'({len(centers)},{len(contexts)})'
            self.centers = centers
            self.contexts = []
            for context in contexts:
                self.contexts.append([c[0] for c in context])
            if use_distance_weight:
                self.coappearence_matrix,self.coappearence_matrix_2 = coappearence_matrix
            else:
                self.coappearence_matrix = coappearence_matrix
            self.tag = use_distance_weight

        def __getitem__(self, index):
            center = self.centers[index]
            contexts = self.contexts[index]
            if self.tag:
                label = self.coappearence_matrix_2[center,contexts].tolist()
            else:
                label = self.coappearence_matrix[center,contexts].tolist()
            if not self.tag:
                return (self.centers[index], self.contexts[index],label)
            return (self.centers[index], self.contexts[index],label,self.coappearence_matrix[center,contexts].tolist())

        def __len__(self):
            return len(self.centers)
    
    if not use_distance_weight:
        ptb_dataset = PTB_DATASET(all_centers,all_contexts,use_distance_weight,coappearence_matrix)
        dataloader = DataLoader(
            ptb_dataset, batch_size=bs, shuffle=True,
            collate_fn=batchify, num_workers=4)
    else:
        ptb_dataset = PTB_DATASET(all_centers,all_contexts,use_distance_weight,(coappearence_matrix,coappearence_matrix_2))
        dataloader = DataLoader(
            ptb_dataset, batch_size=bs, shuffle=True,
            collate_fn=batchify2, num_workers=4)
    
    return dataloader,vocab

In [6]:
def init_weights(m,mode='zero'):
    if type(m) == nn.Embedding:
        if mode =='xavier':
            nn.init.xavier_normal_(m.weight)
        else:
            nn.init.zeros_(m.weight)

class GloVe(nn.Module):
    def __init__(self,vocab_size,embedding_size) -> None:
        super(GloVe,self).__init__()
        self.context = nn.Embedding(vocab_size,embedding_size)
        self.center = nn.Embedding(vocab_size,embedding_size)
        self.context_bias = nn.Embedding(vocab_size,1)
        self.center_bias = nn.Embedding(vocab_size,1)
        init_weights(self.context,mode='xavier')
        init_weights(self.center,mode='xavier')
        init_weights(self.center_bias)
        init_weights(self.context_bias)

    
    def forward(self,center, all_contexts): # input.shape: B,N,vocab_size
        bs,max_len = all_contexts.shape
        contexts = self.context(all_contexts) # shape (B,N_context,embedding_size)
        centers = self.center(center) # shape (B,1,embedding_size)
        contexts_bias = self.context_bias(all_contexts) # shape (B,N_context,1)
        centers_bias = self.center_bias(center) # shape (B,1,1)
        similarity = centers @ contexts.transpose(1,2) # shape:[B,1,N_context]
        similarity = similarity.reshape(bs,max_len)
        centers_bias = centers_bias.reshape(bs,1)
        contexts_bias = contexts_bias.reshape(bs,max_len)
        output = contexts_bias + similarity + centers_bias
        return output

In [7]:
class AverageMeter:
    def __init__(self):
        self.count = 0
        self.sum = 0.
        self.avg = 0.
    
    def update(self,n,val,multiply=True):
        self.count += n
        if multiply:
            self.sum += n*val
        else:
            self.sum += val
        self.avg = self.sum / self.count

class Log_MSELoss(nn.Module):
    # Log + MSELoss **with mask**
    def __init__(self,c=10,alpha=0.75):
        super().__init__()
        self.c = c
        self.alpha = alpha

    def forward(self, inputs, target_1,target_2, mask):
        log_target = torch.log(target_1)
        loss = ((inputs - log_target)**2) * self.weight_func(target_2) * mask 
        loss = loss.sum(dim=1)
        loss = loss / mask.sum(dim=1)
        loss = loss.mean(dim=0)
        return loss
    
    def weight_func(self,target):
        weight = torch.zeros_like(target)
        weight[target>=self.c] = 1.
        weight[target<self.c] = (target[target<self.c]/self.c)**self.alpha
        return weight

def train_one_epoch(model,optimizer,device,data_iter,loss,use_distance_weight):
    train_loss = AverageMeter()
    for batch in data_iter:
        if use_distance_weight:
            center,contexts,mask,label,weight = [data.to(device) for data in batch]
        else:
            center,contexts,mask,label = [data.to(device) for data in batch]
        pred = model(center,contexts)
        
        if use_distance_weight:
            t_loss = loss(pred,label,weight,mask)
        else:
            t_loss = loss(pred,label,label,mask)
        optimizer.zero_grad()
        t_loss.backward()
        optimizer.step()
        train_loss.update(n=center.shape[0],val=t_loss.item(),multiply=True)
    
    return train_loss.avg

@torch.no_grad()
def get_similar_tokens(model,query_token, k, vocab): 
    '''
    in GloVe, x_{ij} = x_{ji}
    so center_embedding and context_embedding are symmetric in math
    but because the two are not same initialized
    so they have a little difference
    **This difference help the model to be robust**(like one model ensemble method——use different initialization method)
    '''
    model.eval()
    W = model.center.weight.data
    W2 = model.context.weight.data
    W = W + W2
    x = W[vocab[query_token]]
    # 计算余弦相似性。增加1e-9以获得数值稳定性
    cos = torch.mv(W, x) / torch.sqrt(torch.sum(W * W, dim=1) *
                                      torch.sum(x * x) + 1e-9)
    topk = torch.topk(cos, k=k+1)[1].cpu().numpy().astype('int32')
    for i in topk[1:]:  # 删除输入词
        print(f'cosine sim={float(cos[i]):.3f}: {vocab.to_tokens(i)}')


def main(sentences):
    for_mlp = True
    use_distance_weight = True
    bs, max_window_size = 512, 5

    if not for_mlp:
        save_dir = './model_vocab_glove'
    else:
        save_dir = './model_vocab_glove_for_mlp_dw' if use_distance_weight else './model_vocab_glove_for_mlp'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    data_iter, vocab = load_data_ptb(bs, max_window_size,sentences,use_distance_weight=use_distance_weight,load_vocab=True,save_dir=save_dir)

    lr = 1e-4
    num_epochs = 20
    embedding_size_lst = [16,32,64,96]
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    for embedding_size in embedding_size_lst:
        model = GloVe(len(vocab),embedding_size).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        loss = Log_MSELoss().to(device)
        
        loss_decription = 0.
        pbar = trange(num_epochs)
        for i in pbar:
            loss_decription = train_one_epoch(model,optimizer,device,data_iter,loss,use_distance_weight)
            description = f'Epoch: {i+1}  Train_Loss:{loss_decription:.3f}'
            pbar.set_description(description)

        # save model
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        with open(os.path.join(save_dir,f'model_{embedding_size}.pth'),'wb') as f:
            torch.save(model,f)
    
        # with open(os.path.join(save_dir,'token2idx.pkl'),'wb') as f:
        #     pickle.dump(vocab.token2idx,f)
        
        # with open(os.path.join(save_dir,'idx2token.pkl'),'wb') as f:
        #     pickle.dump(vocab.idx2token,f)
        print((model.context.weight.data + model.center.weight.data)[:2,:])

In [8]:
main(sentences=all_sentences)

词表大小为：6690


Epoch: 20  Train_Loss:0.560: 100%|██████████| 20/20 [00:27<00:00,  1.37s/it]


tensor([[-1.4441,  1.4483, -1.4588, -1.4445, -1.4577, -1.4462, -1.4425, -1.4295,
         -1.4579, -1.4412,  1.4518, -1.4610,  1.4581, -1.4506,  1.4491, -1.4515],
        [-1.2886,  1.2937, -1.2390, -1.3264, -1.2325, -1.2891, -1.2800, -1.2687,
         -1.2489, -1.3152,  1.2468, -1.2409,  1.2261, -1.2732,  1.2672, -1.2608]],
       device='cuda:0')


Epoch: 20  Train_Loss:0.391: 100%|██████████| 20/20 [00:29<00:00,  1.47s/it]


tensor([[ 1.1010,  1.1176,  1.1196,  1.1208, -1.1125,  1.1225,  1.1094,  1.1173,
          1.1097,  1.1097,  1.1305, -1.1171, -1.1195,  1.1175, -1.1098, -1.1200,
          1.1228, -1.1108,  1.1209, -1.1238, -1.1287, -1.1086, -1.1093, -1.1236,
          1.1143,  1.1261, -1.1137, -1.1078,  1.1223, -1.1071,  1.1168,  1.1119],
        [ 0.9426,  0.9008,  0.8807,  0.9026, -0.9156,  0.8871,  0.9595,  0.8916,
          0.9273,  0.9106,  0.8688, -0.9312, -0.8899,  0.9113, -0.9063, -0.9167,
          0.9229, -0.9157,  0.8989, -0.8860, -0.8995, -0.9329, -0.9078, -0.8758,
          0.9455,  0.8740, -0.9222, -0.9161,  0.9076, -0.9589,  0.9082,  0.9182]],
       device='cuda:0')


Epoch: 20  Train_Loss:0.377: 100%|██████████| 20/20 [00:27<00:00,  1.35s/it]


tensor([[-0.7982,  0.8012,  0.8044,  0.7948,  0.7982,  0.8015, -0.7968, -0.7921,
         -0.7997, -0.7920,  0.8124, -0.7905,  0.8016, -0.7939, -0.7950, -0.7900,
          0.7959, -0.7885,  0.7945, -0.8077,  0.8030,  0.7922, -0.8011,  0.8012,
         -0.8092, -0.7965, -0.7950,  0.7957,  0.7973, -0.7963, -0.8014,  0.7890,
          0.8018, -0.8003,  0.7998, -0.7942,  0.8041,  0.8022, -0.7984, -0.7994,
         -0.7981, -0.7919, -0.7909, -0.7974, -0.7893, -0.8071, -0.8017, -0.7994,
          0.7923, -0.7945, -0.7983, -0.7917,  0.7966,  0.7968,  0.7940, -0.8035,
          0.8097, -0.8041,  0.7994, -0.8050,  0.7955,  0.7979, -0.8024, -0.7959],
        [-0.6435,  0.6203,  0.6171,  0.6874,  0.6332,  0.6296, -0.6724, -0.6831,
         -0.6689, -0.7053,  0.6231, -0.6753,  0.6398, -0.6758, -0.6578, -0.6624,
          0.6746, -0.6840,  0.6808, -0.6581,  0.6264,  0.6651, -0.6485,  0.6458,
         -0.5755, -0.6417, -0.6580,  0.6423,  0.6452, -0.6419, -0.6538,  0.6920,
          0.6252, -0.6662, 

Epoch: 20  Train_Loss:0.363: 100%|██████████| 20/20 [00:27<00:00,  1.36s/it]

tensor([[-0.6618,  0.6528, -0.6595, -0.6607, -0.6642,  0.6548,  0.6448,  0.6608,
          0.6509,  0.6568, -0.6456,  0.6564, -0.6576,  0.6438, -0.6531,  0.6513,
         -0.6543,  0.6554,  0.6571,  0.6561, -0.6558,  0.6569,  0.6453,  0.6568,
          0.6518, -0.6528, -0.6639, -0.6516,  0.6492, -0.6592, -0.6553, -0.6447,
         -0.6525, -0.6591, -0.6541,  0.6161, -0.6580,  0.6561, -0.6612, -0.6452,
         -0.6565, -0.6528, -0.6609, -0.6534,  0.6544,  0.6494,  0.6543,  0.6519,
         -0.6634,  0.6481, -0.6533, -0.6516,  0.6505, -0.6547,  0.6581,  0.6468,
          0.6531,  0.6521, -0.6563, -0.6544,  0.6553,  0.6624, -0.6537,  0.6619,
          0.6516, -0.6589, -0.6552,  0.6592, -0.6490, -0.6566,  0.6571, -0.6532,
          0.6566, -0.6555, -0.6566,  0.6684, -0.6583, -0.6550,  0.6478, -0.6618,
          0.6627,  0.6566,  0.6673,  0.6609, -0.6561,  0.6412,  0.6554,  0.6619,
          0.6529,  0.6607,  0.6454,  0.6506,  0.6534, -0.6511, -0.6420, -0.6527],
        [-0.5012,  0.5530, 